#Generating CaseLaw summaries

In [14]:
%pip install -U -q google-cloud-aiplatform[evaluation]
%pip install -U -q datasets
%pip install -U -q evaluate
%pip install -U -q vertexai
!pip install --upgrade google-genai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==1.71.1, but you have google-cloud-aiplatform 1.109.0 which is incompatible.


In [35]:
from datasets import load_dataset
import pandas as pd
from google.cloud import storage
import json
from tqdm import tqdm


In [36]:
from datasets import Dataset
from itertools import islice

# Initialize GCS client
client = storage.Client()
bucket_name = "finetune-data123"
prefix = "task2025_train"  # folder in the bucket
bucket = client.bucket(bucket_name)

# List all .txt blobs in the folder
blobs = bucket.list_blobs(prefix=prefix)
data = []
i = 1

for blob in blobs:
    print(f"{i}")
    if blob.name.endswith(".txt"):
        i +=1
        # Download the content as string
        text = blob.download_as_text()
        # Keep file name and content
        file_name = blob.name.split("/")[-1]   # '000002.txt'
        number = file_name.split(".")[0]       # '000002'
        data.append({"file_name": number, "text": text})

# Convert to Dataset or DataFrame
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

print(dataset)


1
2
Dataset({
    features: ['file_name', 'text'],
    num_rows: 2
})


In [37]:
# Convert DataFrame to JSON string
json_str = df.to_json(orient="records", lines=True)

# Initialize GCS client
client = storage.Client()
bucket_name = "finetune-data123"
bucket = client.bucket(bucket_name)

# Create blob in bucket
blob = bucket.blob("results/task2025_train.json")

# Upload JSON string directly
blob.upload_from_string(json_str, content_type="application/json")

print("Done)")

Done)


In [39]:
# Compute lengths of all texts
lengths = df['text'].apply(len)

# Calculate statistics
average_length = lengths.mean()
min_length = lengths.min()
max_length = lengths.max()

print(f"Average length: {average_length}")
print(f"Minimum length: {min_length}")
print(f"Maximum length: {max_length}")

print(f"Lenght:{len(df)}")


Average length: 41235.5
Minimum length: 32929
Maximum length: 49542
Lenght:2


In [40]:

print("Length of dataframe:", len(df))
print("Keys (columns):", df.columns.tolist())


Length of dataframe: 2
Keys (columns): ['file_name', 'text']


In [41]:
from google import genai
from google.genai import types
import base64
import time
from google.api_core import exceptions as g_exceptions

def generate_finetuned(text, retries=3, backoff=2):
    client = genai.Client(
        vertexai=True,
        project="651136027056",
        location="europe-west9",
    )

    model = "projects/651136027056/locations/europe-west9/endpoints/3743955839833079808"

    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=text)]
        )
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=0.0,
        top_p=0.95,
        max_output_tokens=5000,
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
        thinking_config=types.ThinkingConfig(thinking_budget=0),
    )

    attempt = 0
    while attempt < retries:
        try:
            output_text = ""
            for chunk in client.models.generate_content_stream(
                model=model,
                contents=contents,
                config=generate_content_config,
            ):
                if chunk and getattr(chunk, "text", None):
                    output_text += chunk.text

            if not output_text.strip():
                raise ValueError("Empty response from model")

            return output_text  # ✅ Success, exit function

        except (g_exceptions.GoogleAPICallError, g_exceptions.RetryError,
                ValueError, Exception) as e:
            attempt += 1
            print(f"[Error] Attempt {attempt}/{retries} failed: {e}")

            if attempt < retries:
                sleep_time = backoff ** attempt
                print(f"Retrying in {sleep_time}s...")
                time.sleep(sleep_time)
            else:
                print("[Error] Max retries exceeded.")
                return "[Model error: no response]"


In [42]:
from tqdm import tqdm
import pandas as pd
from google.cloud import storage
import json

# Initialize GCS client and bucket
client = storage.Client()
bucket_name = "finetune-data123"
bucket = client.bucket(bucket_name)
results_prefix = "results"  # folder in bucket for results

def save_to_gcs(bucket, folder, file_name, data):
    """Save a dictionary as JSON to GCS."""
    blob = bucket.blob(f"{folder}/{file_name}")
    blob.upload_from_string(json.dumps(data, ensure_ascii=False), content_type="application/json")

def generate_summaries(df, generate_fn):
    """
    df: DataFrame with columns 'file_name' and 'text'
    generate_fn: function to generate text from model (e.g., generate_finetuned)
    """
    summaries = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        file_name = row["file_name"]
        case_text = "Summarize the following document: " + row["text"]
        summary = generate_fn(case_text)
        if summary == "[Model error: no response]":
            print(f"⚠️ Model returned no response for file {file_name}")

        record = {
            "file_name": file_name,
            "text": case_text,
            "summary": summary
        }

        # Save each summary immediately to GCS
        json_file_name = f"{file_name}.json"
        save_to_gcs(bucket, results_prefix, json_file_name, record)

        summaries.append(record)
        #else skip


    return pd.DataFrame(summaries)

In [43]:
# 3️⃣ Generate summaries using finetuned model
summaries_df = generate_summaries(df, generate_finetuned)


100%|██████████| 2/2 [00:09<00:00,  4.85s/it]


OSError: Cannot save file into a non-existent directory: 'results'

In [44]:
from google.cloud import storage


# Convert DataFrame to JSONL string
jsonl_str = summaries_df.to_json(orient="records", lines=True, force_ascii=False)

# Initialize GCS client
client = storage.Client()
bucket_name = "finetune-data123"
bucket = client.bucket(bucket_name)

# Create blob in bucket (path in bucket)
blob = bucket.blob("results/cleaned_summaries.jsonl")

# Upload JSONL string directly
blob.upload_from_string(jsonl_str, content_type="application/json")

print(f"✅ Saved summaries for {len(summaries_df)} cases to results/cleaned_summaries.jsonl in GCS")


✅ Saved summaries for 2 cases to results/cleaned_summaries.jsonl in GCS
